In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

## CODE for growth curves 5a-5h

#### Run the experiment notebooks for the following notebooks
1. ../expt_notebooks/EXP-22-BV6194 kleb and aureus CIP.ipynb
2. ../expt_notebooks/EXP-22-BV6196 Pseudomonas and Acinitobacter Gentamycin 4.ipynb
3. ../expt_notebooks/EXP-22-BV6197 Ecoli and pmirbalis Nitrofurantoin 32ugml.ipynb
4. ../expt_notebooks/EXP-22-BZ0300 Ecoli and Efaecalis Vancomycin 4.ipynb

and their repeats

5. ../expt_notebooks/EXP-22-BZ0306 Pseudomonas and Acinitobacter Gentamycin 4.ipynb
6. ../expt_notebooks/EXP-22-BZ0307 Ecoli and Efaecalis Vancomycin.ipynb
7. ../expt_notebooks/EXP-22-BZ0312 kleb and aureus CIP.ipynb
8. ../expt_notebook/EXP-22-BZ0313 Ecoli and pmirbalis Nitrofurantoin 32ugml.ipynb

In [10]:
from narsil2.mm.growth import growth_rate_pickles
from pathlib import Path
import numpy as np
%matplotlib qt

In [11]:
species_full_names = {
    'ecoli':'E.coli',
    'kpneumoniae':'K.pneumoniae',
    'paeruginosa':'P.aeruginosa',
    'efaecalis':'E.faecalis',
    'abaumanii':  'A.baumannii',
    'pmirbalis': 'P.mirabilis',
    'saureus': 'S.aureus',
}

# color scheme use good color wisely that looks good
labels_to_colors = {
    0: 'deeppink', # E.coli
    1: 'gold', # k.pneumoniae
    2: 'peru', # p.aeruginosa
    3: 'orangered', # e.faecalis
    4: 'teal', # A.baumanni
    5: 'green', # p.mirabilis
    6: 'royalblue', # s.aureus 
    7: 'k' # background
}

In [12]:
root_dir = '/mnt/sda2/Praneeth/ASTFISH/'

#### First experiment K.pneumoniae and S.aureus Ciprofloxacin 1ug/ml

In [18]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 181))
Ab_Positions = list(range(201, 281))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BV6194 AST FISH 220210/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=1,
                        antibiotic_name='Ciprofloxacin',
                       n_frames=30)
color_scheme = {
    'kpneumoniae': 'gold',
    'saureus': 'royalblue',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['ecoli','paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis'],
                              species_full_name=species_full_names)


In [19]:
def get_raw_data(g, species):
    if species != 'pool_Ab' and species != 'pool_no_Ab':
        print(f'{species} Normalized growth rates Ab response, sem of growth rate, time')
        species_noab = g.No_Ab_Clean_GrowthRates[species]
        species_ab = g.Ab_Clean_GrowthRates[species]
        normalized_growth_rates = species_ab[0]/species_noab[0]
        species_error_ab = species_ab[1]/species_noab[0]/np.sqrt(species_ab[2])
        return (normalized_growth_rates, species_error_ab, np.array(list(range(0, 2 * g.n_frames, 2))))
    elif species == 'pool_no_Ab':
        print(f'{species} ones, sem of pooled growth rate, time')

        noAb_pool = g.No_Ab_Clean_Pooled_GrowthRates
        Ab_pool = g.Ab_Clean_Pooled_GrowthRates
        ones = np.array([1] * g.n_frames)
        noAb_pool_err = noAb_pool[1]/ noAb_pool[0]/ np.sqrt(noAb_pool[2])
        return (ones, noAb_pool_err, np.array(list(range(0, 2 * g.n_frames, 2))))
    
    elif species == 'pool_Ab':
        print(f'{species} normalized growth rates, sem of pooled growth rate, time')
        noAb_pool = g.No_Ab_Clean_Pooled_GrowthRates
        Ab_pool = g.Ab_Clean_Pooled_GrowthRates
        normalized_pool = Ab_pool[0]/noAb_pool[0]
        Ab_pool_err = Ab_pool[1]/noAb_pool[0]/np.sqrt(Ab_pool[2])
        return (normalized_pool, Ab_pool_err, np.array(list(range(0, 2 * g.n_frames, 2))))

In [20]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [21]:
get_raw_data(g, 'kpneumoniae')

kpneumoniae Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.97609907,
        0.9694505 , 0.95404448, 0.93703801, 0.9238636 , 0.91039752,
        0.90100401, 0.89762065, 0.89661915, 0.88972933, 0.88483562,
        0.87395531, 0.86144261, 0.85038823, 0.83450507, 0.82171131,
        0.81039842, 0.79839936, 0.78840173, 0.77952927, 0.76476303,
        0.74628842, 0.7316837 , 0.71304766, 0.69948245, 0.68703443]),
 array([       nan,        nan,        nan,        nan, 0.0049115 ,
        0.00450865, 0.0043625 , 0.00414463, 0.0039683 , 0.00372789,
        0.0034935 , 0.00334165, 0.00312485, 0.00294449, 0.00286487,
        0.00273912, 0.00252996, 0.00240343, 0.00241318, 0.00238468,
        0.00243172, 0.0024543 , 0.0025014 , 0.00253731, 0.002567  ,
        0.00254783, 0.00254648, 0.00257233, 0.00261734, 0.0026538 ]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [22]:
get_raw_data(g, 'saureus')

saureus Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.00258063,
        1.02193078, 0.99134906, 1.03119406, 1.0296968 , 0.97467086,
        1.02224604, 1.02021959, 1.03433401, 1.07625364, 1.06301936,
        1.0553116 , 1.00962699, 1.01633834, 0.97558848, 0.93757928,
        0.90835559, 0.88269522, 0.87919898, 0.86761991, 0.84127027,
        0.8235501 , 0.81023578, 0.78843244, 0.78664392, 0.77193218]),
 array([       nan,        nan,        nan,        nan, 0.02492636,
        0.02573295, 0.02330621, 0.02450351, 0.02322336, 0.02037347,
        0.02116434, 0.02051942, 0.01931401, 0.01994824, 0.01793661,
        0.01707768, 0.01627196, 0.0162103 , 0.01427746, 0.01268487,
        0.01151973, 0.01089633, 0.01070296, 0.01040026, 0.00958631,
        0.00874355, 0.00809333, 0.00769921, 0.0074564 , 0.00709115]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [23]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 1.00670781,
        1.00407091, 0.98368516, 0.98065657, 0.96934179, 0.94177305,
        0.94864417, 0.94599386, 0.95166632, 0.96198849, 0.95607917,
        0.94389118, 0.92082821, 0.91408355, 0.89047352, 0.8677384 ,
        0.84974824, 0.8338175 , 0.82456515, 0.81441889, 0.79381318,
        0.77578317, 0.76216085, 0.7434401 , 0.73425572, 0.7224108 ]),
 array([       nan,        nan,        nan,        nan, 0.00757125,
        0.00755176, 0.00703841, 0.0071881 , 0.00695162, 0.0063827 ,
        0.006519  , 0.00648615, 0.00611613, 0.00617978, 0.00566805,
        0.00551351, 0.00541368, 0.00543743, 0.00495249, 0.00458715,
        0.00432509, 0.00421768, 0.00427187, 0.00425394, 0.00410311,
        0.00386129, 0.00367404, 0.00358939, 0.00354884, 0.00350316]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [24]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.00766087,
        0.00737415, 0.00745515, 0.00728096, 0.00708651, 0.00703478,
        0.00660915, 0.00668336, 0.00669372, 0.00653477, 0.00604907,
        0.00590725, 0.00602585, 0.00572606, 0.00570346, 0.00564369,
        0.00572102, 0.00568834, 0.00563567, 0.0054782 , 0.00554244,
        0.00539295, 0.00521566, 0.00512392, 0.00488444, 0.00493892]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

#### Second experiment P.aeruginosa and A.baumannii Gentamycin 2ug/ml

In [25]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 181))
Ab_Positions = list(range(201, 281))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BV6196 AST FISH 220214/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}

g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=2,
                        antibiotic_name='Gentamycin',
                       n_frames=30)

color_scheme = {
    'paeruginosa': 'peru',
    'abaumanii': 'teal',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['ecoli','kpneumoniae' , 'efaecalis', 'pmirbalis', 'saureus'],
                              species_full_name=species_full_names)

In [26]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [27]:
get_raw_data(g, 'paeruginosa')

paeruginosa Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.01950908,
        1.01820827, 1.03125615, 0.90833021, 0.81692626, 0.78189223,
        0.68969709, 0.60877579, 0.54417026, 0.51499515, 0.53458909,
        0.4817754 , 0.44535095, 0.42270309, 0.4106725 , 0.42176539,
        0.44481911, 0.4691332 , 0.46131785, 0.4402301 , 0.41034757,
        0.4100294 , 0.40608097, 0.38434927, 0.40762165, 0.34736822]),
 array([       nan,        nan,        nan,        nan, 0.03577916,
        0.04335259, 0.05486387, 0.05215435, 0.04649838, 0.04544628,
        0.04124864, 0.03841541, 0.04538007, 0.04844494, 0.05571949,
        0.02649099, 0.02407104, 0.02054446, 0.02282922, 0.02334313,
        0.02335387, 0.02188214, 0.02220194, 0.01902979, 0.01791255,
        0.01754091, 0.02033862, 0.01769769, 0.02429198, 0.01863847]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [28]:
get_raw_data(g, 'abaumanii')

abaumanii Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.09269805,
        1.06695057, 1.03746002, 1.02838435, 1.02024724, 1.00728565,
        0.99276342, 0.97945698, 0.97835583, 0.98546604, 0.97340565,
        0.97625911, 0.96364623, 0.9535524 , 0.95686597, 0.94425807,
        0.9546289 , 0.94327351, 0.93680696, 0.94003354, 0.93744402,
        0.93094607, 0.91741359, 0.91297582, 0.91003419, 0.91710677]),
 array([       nan,        nan,        nan,        nan, 0.00669725,
        0.00638454, 0.00644862, 0.00621542, 0.00605089, 0.00600847,
        0.00585838, 0.00569358, 0.00550942, 0.00542085, 0.00540116,
        0.00541998, 0.00535288, 0.00514891, 0.00514394, 0.00507446,
        0.00507139, 0.00496964, 0.00493834, 0.00486374, 0.00466232,
        0.00455769, 0.00445232, 0.0043946 , 0.00434611, 0.00431396]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [29]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.00690489,
        0.00701992, 0.00686206, 0.00657033, 0.0060064 , 0.00642357,
        0.00595567, 0.00598144, 0.00588008, 0.00573304, 0.00547937,
        0.00538375, 0.00538176, 0.00530162, 0.00505991, 0.00481476,
        0.00472327, 0.00455015, 0.00442029, 0.00431191, 0.00434225,
        0.00414091, 0.00404173, 0.00409916, 0.00412576, 0.00406202]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [30]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 1.0886732 ,
        1.066743  , 1.0390476 , 1.02262402, 1.00791038, 0.99406384,
        0.97629866, 0.95917018, 0.95295088, 0.95806045, 0.94837508,
        0.95129404, 0.93694302, 0.92551146, 0.9290103 , 0.91698919,
        0.93024378, 0.92093801, 0.91458584, 0.91700405, 0.91528756,
        0.91097028, 0.89720353, 0.89101797, 0.89024003, 0.89032462]),
 array([       nan,        nan,        nan,        nan, 0.00663248,
        0.00647852, 0.0067496 , 0.00664405, 0.00650087, 0.00640635,
        0.00627488, 0.00611559, 0.0062329 , 0.00629085, 0.00641249,
        0.00586156, 0.00575475, 0.00550694, 0.0055618 , 0.00548156,
        0.00542069, 0.00523994, 0.00524119, 0.00513571, 0.00499781,
        0.00488627, 0.00480495, 0.00470775, 0.00470744, 0.00464655]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

#### Third experiment E.coli and P.mirabilis Nitrofurantoin 32ug/ml

In [31]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 165))
Ab_Positions = list(range(201, 265))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BV6197 AST FISH 220215/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates', # this name will also be used in growth rates
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 20,
    'flip_positions': range(201, 265),
    'num_channels': 42,
    'growth_type': 'ratio'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=32,
                        antibiotic_name='Nitrofurantoin',
                       n_frames=30)
color_scheme = {
    'ecoli': 'deeppink',
    'pmirbalis': 'green',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['paeruginosa','kpneumoniae' , 'efaecalis', 'abaumanii', 'saureus'],
                              species_full_name=species_full_names)

/home/pk/anaconda3/envs/narsil2_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pk/anaconda3/envs/narsil2_env/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [32]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [33]:
get_raw_data(g, 'ecoli')

ecoli Normalized growth rates Ab response, sem of growth rate, time


(array([       nan, 0.        , 0.        , 0.        , 0.33352438,
        0.31594157, 0.32506956, 0.29777847, 0.28842325, 0.26355473,
        0.25169537, 0.23480629, 0.22180184, 0.21934948, 0.21272315,
        0.19865545, 0.19419132, 0.19100491, 0.19135717, 0.18248583,
        0.17718804, 0.17676409, 0.17300334, 0.17177269, 0.16013456,
        0.16154973, 0.1639654 , 0.16185112, 0.16034222, 0.17970062]),
 array([       nan,        nan,        nan,        nan, 0.00878643,
        0.00883505, 0.00932055, 0.0073698 , 0.00791713, 0.00593228,
        0.00597833, 0.00560947, 0.00511221, 0.00477649, 0.00451662,
        0.00419106, 0.00475314, 0.00420399, 0.00408499, 0.003806  ,
        0.00378919, 0.00495802, 0.00551736, 0.00590522, 0.00425278,
        0.00487866, 0.00459388, 0.00452858, 0.00439414, 0.00578212]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [34]:
get_raw_data(g, 'pmirbalis')

pmirbalis Normalized growth rates Ab response, sem of growth rate, time


(array([       nan, 0.        , 0.        , 0.        , 0.77972091,
        0.79295179, 0.84720698, 0.78052143, 0.7708711 , 0.74149584,
        0.74187308, 0.75861482, 0.74271233, 0.74784797, 0.72739132,
        0.69215456, 0.69052973, 0.68433835, 0.68182098, 0.64601564,
        0.62209684, 0.61120658, 0.59726169, 0.60270153, 0.57566688,
        0.5703814 , 0.56740428, 0.5638127 , 0.56548998, 0.54651221]),
 array([       nan,        nan,        nan,        nan, 0.0078698 ,
        0.00726116, 0.00758508, 0.00636822, 0.00589993, 0.00537808,
        0.00549383, 0.00529866, 0.00523264, 0.00519954, 0.004836  ,
        0.0046663 , 0.00469651, 0.00494821, 0.00495738, 0.00481117,
        0.00495211, 0.00455165, 0.00470527, 0.00465768, 0.00430168,
        0.00461394, 0.00494611, 0.00485336, 0.00513779, 0.00492114]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [35]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan, 0.17235309, 0.14976336, 0.22944893, 0.00748222,
        0.00721946, 0.00674527, 0.00754908, 0.00641576, 0.00544997,
        0.0048942 , 0.0050214 , 0.0046277 , 0.00453385, 0.00392696,
        0.00379433, 0.00389756, 0.00380763, 0.00379736, 0.00388629,
        0.00391713, 0.00393937, 0.0039351 , 0.00391513, 0.00369746,
        0.0037046 , 0.00364011, 0.00366445, 0.00369477, 0.0036124 ]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [36]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan, 0.        , 0.        , 0.        , 0.59722732,
        0.60647245, 0.63833413, 0.59730086, 0.58662029, 0.55903508,
        0.550214  , 0.54051138, 0.5225945 , 0.52198933, 0.50589786,
        0.47834183, 0.47001327, 0.45950327, 0.45703348, 0.43393334,
        0.41901567, 0.41141467, 0.40063478, 0.39930059, 0.37975082,
        0.37751755, 0.37638374, 0.37202925, 0.36961804, 0.37177107]),
 array([       nan,        nan,        nan,        nan, 0.00697672,
        0.00696328, 0.00743432, 0.00652227, 0.00646142, 0.00584182,
        0.00585393, 0.00570872, 0.00548126, 0.00538988, 0.00514055,
        0.00482847, 0.00483526, 0.00467442, 0.00457021, 0.0043298 ,
        0.00425808, 0.00426916, 0.0043562 , 0.00440131, 0.00382117,
        0.00400108, 0.00396985, 0.00387258, 0.00389254, 0.00405555]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

#### Fourth Experiment E.coli and E.faecalis Vancomycin 4ug/ml

In [37]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 163))
Ab_Positions = list(range(201, 263))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BZ0300 AST FISH 220422/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=4,
                        antibiotic_name='Vancomycin',
                       n_frames=29)

color_scheme = {
    'ecoli': 'deeppink',
    'efaecalis': 'orangered',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['paeruginosa','kpneumoniae' , 'pmirbalis', 'abaumanii', 'saureus'],
                              species_full_name=species_full_names)

In [38]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [39]:
get_raw_data(g, 'ecoli')

ecoli Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.00184255,
        1.03471926, 0.99037229, 0.9769223 , 0.94687819, 0.97551233,
        0.95457613, 0.96926984, 0.99108963, 1.00102551, 1.01176297,
        0.98008906, 0.98703852, 0.97127655, 0.96057858, 0.97588483,
        0.97867731, 0.98397292, 0.99962111, 0.98113293, 1.01143845,
        1.00485449, 1.00072316, 0.98624003, 0.98036639]),
 array([       nan,        nan,        nan,        nan, 0.01152346,
        0.01255146, 0.00983323, 0.0091321 , 0.00793598, 0.00779799,
        0.00762884, 0.00774101, 0.00768603, 0.00734039, 0.007247  ,
        0.00721547, 0.00686205, 0.00652972, 0.00645185, 0.00652257,
        0.00635836, 0.0063835 , 0.00640543, 0.00607082, 0.00604542,
        0.00595315, 0.005946  , 0.00589186, 0.00550279]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [40]:
get_raw_data(g, 'efaecalis')

efaecalis Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.59967154,
        0.54322916, 0.51626114, 0.53703397, 0.55430281, 0.5700886 ,
        0.57236737, 0.56806246, 0.55597071, 0.52450775, 0.47927177,
        0.43203666, 0.41106237, 0.37002495, 0.34829525, 0.33351582,
        0.32780882, 0.34303386, 0.32990371, 0.34560103, 0.32843569,
        0.33929925, 0.33577597, 0.32834016, 0.30624066]),
 array([       nan,        nan,        nan,        nan, 0.01610125,
        0.01709464, 0.01516148, 0.01442909, 0.01470202, 0.01381164,
        0.0155203 , 0.01556906, 0.01634662, 0.01619855, 0.02192951,
        0.02303919, 0.02392464, 0.01873588, 0.01888909, 0.01970933,
        0.02016396, 0.0207577 , 0.02015566, 0.02098755, 0.02058991,
        0.0216259 , 0.02103028, 0.01996085, 0.01840088]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [41]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.01157785,
        0.01033818, 0.01021311, 0.01008812, 0.00932571, 0.00877961,
        0.00869542, 0.00846747, 0.00751427, 0.00706653, 0.00747007,
        0.0075138 , 0.00741829, 0.00694496, 0.00696772, 0.00729217,
        0.00672822, 0.00652962, 0.00607695, 0.00601811, 0.00562506,
        0.00557329, 0.0054193 , 0.00546815, 0.00518899]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [42]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 0.91594862,
        0.91819531, 0.88563787, 0.8905836 , 0.88890691, 0.92018989,
        0.91097475, 0.93179046, 0.95231177, 0.97494442, 0.98682344,
        0.96042757, 0.97196797, 0.95407955, 0.94509547, 0.95613378,
        0.95877302, 0.97097012, 0.97772645, 0.96487744, 0.98569761,
        0.98966098, 0.98647958, 0.97359036, 0.95989317]),
 array([       nan,        nan,        nan,        nan, 0.01107447,
        0.01202991, 0.01021866, 0.00963642, 0.00887569, 0.00880259,
        0.00886951, 0.00913673, 0.00933382, 0.00942565, 0.01010487,
        0.00997464, 0.00990969, 0.00934865, 0.00928997, 0.00926652,
        0.0090156 , 0.00884095, 0.00887767, 0.00844786, 0.00853443,
        0.00833935, 0.00826195, 0.00811373, 0.00772587]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

### Repeats

#### Pseudomonas Baumanii


In [48]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 177))
Ab_Positions = list(range(201, 277))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BZ0306 AST FISH 220621/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}

g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=2,
                        antibiotic_name='Gentamycin',
                       n_frames=30)

color_scheme = {
    'paeruginosa': 'peru',
    'abaumanii': 'teal',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['ecoli','kpneumoniae' , 'efaecalis', 'pmirbalis', 'saureus'],
                              species_full_name=species_full_names)

In [49]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [50]:
get_raw_data(g, 'paeruginosa')

paeruginosa Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.96103111,
        1.01799181, 0.99183109, 0.93979562, 0.90653469, 0.84886248,
        0.80475918, 0.71710077, 0.66092748, 0.62464815, 0.60050542,
        0.54710312, 0.51315857, 0.4971183 , 0.47155114, 0.47520696,
        0.45362929, 0.44271856, 0.43960576, 0.43492909, 0.4270916 ,
        0.41395985, 0.39860267, 0.41235699, 0.44062401, 0.45191059]),
 array([       nan,        nan,        nan,        nan, 0.01931788,
        0.02162757, 0.02020165, 0.02121302, 0.02476449, 0.02276752,
        0.0207374 , 0.0186064 , 0.0153176 , 0.01380092, 0.01501948,
        0.01261174, 0.01290888, 0.01358916, 0.01205697, 0.01283033,
        0.01221531, 0.01219013, 0.01066433, 0.01165739, 0.01021797,
        0.00988933, 0.00942838, 0.0104397 , 0.00993746, 0.01147793]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [51]:
get_raw_data(g, 'abaumanii')

abaumanii Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.0267024 ,
        1.00153879, 0.97384812, 0.96068135, 0.95599149, 0.96685807,
        0.96006683, 0.95315497, 0.96402186, 0.96549794, 0.9533482 ,
        0.9422582 , 0.92925549, 0.94366731, 0.92803313, 0.91416543,
        0.91443562, 0.90402263, 0.91409777, 0.90234063, 0.90617168,
        0.89862022, 0.89028047, 0.90061961, 0.90823842, 0.89994526]),
 array([       nan,        nan,        nan,        nan, 0.01208803,
        0.01228992, 0.01204812, 0.0123545 , 0.01239663, 0.01197661,
        0.01103335, 0.00994744, 0.00976587, 0.00970614, 0.00927237,
        0.00937124, 0.00878747, 0.0088154 , 0.00855917, 0.00838688,
        0.00845184, 0.00823572, 0.00806907, 0.0079574 , 0.00764609,
        0.00770594, 0.00746919, 0.00743859, 0.0071516 , 0.00688545]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [52]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.01047685,
        0.00940889, 0.00841987, 0.00819537, 0.00814341, 0.00804245,
        0.007696  , 0.00736341, 0.00747159, 0.00680642, 0.00640454,
        0.00651744, 0.00634745, 0.00625793, 0.00617199, 0.0061809 ,
        0.00601765, 0.00609402, 0.00575951, 0.00546639, 0.00528733,
        0.00520608, 0.00510901, 0.0050799 , 0.00489177, 0.0051337 ]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [53]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 0.99896385,
        1.00432204, 0.97571425, 0.94982185, 0.93452645, 0.9193034 ,
        0.89785212, 0.85874683, 0.83601307, 0.82179258, 0.80376193,
        0.77470359, 0.75166708, 0.75370355, 0.73542434, 0.7289939 ,
        0.72390477, 0.71336706, 0.72479538, 0.71880232, 0.71806086,
        0.70988657, 0.70525251, 0.71204957, 0.72328537, 0.71810388]),
 array([       nan,        nan,        nan,        nan, 0.0099581 ,
        0.01048009, 0.0099949 , 0.01042553, 0.01128802, 0.01090408,
        0.01015311, 0.0094712 , 0.00883967, 0.00852722, 0.00877543,
        0.00833598, 0.00833935, 0.00860235, 0.00815798, 0.00817442,
        0.00817424, 0.00809753, 0.00771291, 0.00786074, 0.00753098,
        0.00744398, 0.00729625, 0.00739922, 0.00703501, 0.00717565]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

#### Ecoli and Efaecalis

In [54]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 171))
Ab_Positions = list(range(201, 271))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BZ0307 AST FISH 220622/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=4,
                        antibiotic_name='Vancomycin',
                       n_frames=29)

color_scheme = {
    'ecoli': 'deeppink',
    'efaecalis': 'orangered',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['paeruginosa','kpneumoniae' , 'pmirbalis', 'abaumanii', 'saureus'],
                              species_full_name=species_full_names)

In [55]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [56]:
get_raw_data(g, 'ecoli')

ecoli Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.97057886,
        0.95836806, 0.98002046, 0.97345568, 0.97205462, 0.97992873,
        0.96144847, 0.96136053, 0.96058473, 0.96263763, 0.96324099,
        0.96012292, 0.95387916, 0.94747416, 0.95020023, 0.94372054,
        0.95019212, 0.93479295, 0.93587045, 0.9356638 , 0.93266284,
        0.92766406, 0.93356362, 0.9340857 , 0.92791782]),
 array([       nan,        nan,        nan,        nan, 0.00640933,
        0.00639932, 0.00632531, 0.00677593, 0.00566422, 0.00575337,
        0.00548017, 0.00562612, 0.00551222, 0.00537934, 0.00539519,
        0.00476771, 0.0046569 , 0.00447074, 0.00441906, 0.00434176,
        0.00425577, 0.00394012, 0.00383017, 0.00394756, 0.00405002,
        0.00398576, 0.00368946, 0.00366854, 0.00341773]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [57]:
get_raw_data(g, 'efaecalis')

efaecalis Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.51311029,
        0.491438  , 0.49016082, 0.48216782, 0.44185487, 0.39986209,
        0.37850922, 0.32836975, 0.29792395, 0.27176666, 0.25956697,
        0.2343186 , 0.21936552, 0.2031321 , 0.1968475 , 0.18850108,
        0.17527214, 0.16984692, 0.17710408, 0.18714026, 0.1828314 ,
        0.18540246, 0.17410934, 0.17130196, 0.1690813 ]),
 array([       nan,        nan,        nan,        nan, 0.00913948,
        0.00830565, 0.00763639, 0.00744574, 0.00765545, 0.00781523,
        0.00853467, 0.00824397, 0.00937103, 0.00936591, 0.01055126,
        0.01083558, 0.01078455, 0.01121327, 0.01154865, 0.01101645,
        0.01026308, 0.01010338, 0.01021541, 0.01029439, 0.00994104,
        0.00979309, 0.00961456, 0.01026531, 0.00930201]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [58]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.00714015,
        0.00723715, 0.00669791, 0.00726409, 0.00700621, 0.00648552,
        0.00663002, 0.0061855 , 0.00572458, 0.00534416, 0.0052551 ,
        0.00524504, 0.00527864, 0.00511227, 0.00480414, 0.00467902,
        0.00448742, 0.00461056, 0.00427432, 0.00420967, 0.00409064,
        0.00399656, 0.00391989, 0.00377515, 0.00374548]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

In [59]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 0.86192252,
        0.84085542, 0.84600991, 0.84406719, 0.8342406 , 0.83294715,
        0.8352136 , 0.83240523, 0.84649793, 0.85501394, 0.87408136,
        0.88282031, 0.89069583, 0.89045928, 0.90546116, 0.89687124,
        0.89284867, 0.88255654, 0.8790396 , 0.87845443, 0.87349525,
        0.86737816, 0.86849738, 0.8693644 , 0.85502029]),
 array([       nan,        nan,        nan,        nan, 0.00732262,
        0.00699034, 0.00669248, 0.00688504, 0.00648283, 0.00675328,
        0.00673496, 0.00704314, 0.00719974, 0.00719652, 0.00732533,
        0.00704298, 0.00691794, 0.00681036, 0.00670392, 0.00648943,
        0.0063702 , 0.00605548, 0.00590063, 0.00576866, 0.00572118,
        0.00551702, 0.00519804, 0.00504098, 0.00479568]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]))

#### Klebsiella and S.aureus

In [60]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 162))
Ab_Positions = list(range(201, 262))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BZ0312 AST FISH 220628/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates',
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 15,
    'flip_positions': range(201, 281),
    'num_channels': 32,
    'growth_type': 'rolling'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=1,
                        antibiotic_name='Ciprofloxacin',
                       n_frames=30)
color_scheme = {
    'kpneumoniae': 'gold',
    'saureus': 'royalblue',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['ecoli','paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis'],
                              species_full_name=species_full_names)


In [61]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [62]:
get_raw_data(g, 'kpneumoniae')

kpneumoniae Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.97651297,
        0.97166523, 0.95534324, 0.93838349, 0.92515232, 0.91055335,
        0.90187521, 0.8959952 , 0.89533623, 0.89017381, 0.88639954,
        0.87547629, 0.86425168, 0.85362561, 0.83918346, 0.82571987,
        0.81310093, 0.80163914, 0.79089239, 0.78342378, 0.76813587,
        0.74968512, 0.73575493, 0.71604907, 0.7013482 , 0.6888116 ]),
 array([       nan,        nan,        nan,        nan, 0.00574355,
        0.00524602, 0.00504081, 0.00491436, 0.00451919, 0.00427726,
        0.00400503, 0.00386909, 0.00361523, 0.00340612, 0.00330721,
        0.00313777, 0.00284511, 0.00273578, 0.00278461, 0.00278864,
        0.00283857, 0.00284167, 0.00289489, 0.0029458 , 0.00294439,
        0.00290537, 0.00295007, 0.00296843, 0.00303509, 0.0030848 ]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [63]:
get_raw_data(g, 'saureus')

saureus Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 1.04767789,
        1.08014761, 1.04100811, 1.04410604, 1.03280936, 0.99481995,
        1.0450374 , 1.03755175, 1.03824975, 1.05853503, 1.02744405,
        1.0079362 , 0.9978506 , 1.02782775, 0.98670379, 0.94539019,
        0.91029167, 0.88256639, 0.88778602, 0.88422631, 0.85249349,
        0.83806472, 0.82802169, 0.81204896, 0.80752518, 0.7898705 ]),
 array([       nan,        nan,        nan,        nan, 0.02875759,
        0.03046119, 0.02848051, 0.02868419, 0.02681786, 0.02409914,
        0.02534354, 0.02451092, 0.02231901, 0.02193006, 0.0194037 ,
        0.01825287, 0.01769589, 0.01806708, 0.01626904, 0.01475116,
        0.01355585, 0.01193557, 0.01209228, 0.01213295, 0.01106412,
        0.01043417, 0.00983978, 0.0094486 , 0.00900106, 0.00846871]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [64]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.00844653,
        0.00821218, 0.00823221, 0.00828072, 0.00796574, 0.00800866,
        0.00750187, 0.00747919, 0.00749776, 0.0073011 , 0.00699064,
        0.00697329, 0.00697193, 0.00651114, 0.00642723, 0.00643927,
        0.00657489, 0.00650079, 0.00630581, 0.00608087, 0.00629747,
        0.00609662, 0.0059215 , 0.00588248, 0.00563009, 0.00558707]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [65]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 1.02084189,
        1.02132521, 0.99779854, 0.98425925, 0.96995904, 0.94685024,
        0.95477734, 0.94830944, 0.94941313, 0.95337518, 0.94209285,
        0.92610698, 0.91509441, 0.91599795, 0.89335716, 0.86972882,
        0.84896475, 0.83301292, 0.82826448, 0.82132575, 0.79860217,
        0.7823932 , 0.77054911, 0.75395125, 0.74344978, 0.72891128]),
 array([       nan,        nan,        nan,        nan, 0.00866341,
        0.00880443, 0.0084317 , 0.00847537, 0.00811024, 0.00756387,
        0.00780613, 0.00779406, 0.00713016, 0.00696919, 0.00638408,
        0.0061951 , 0.00609212, 0.00613769, 0.00572938, 0.00539252,
        0.00513963, 0.00475119, 0.00482329, 0.00492727, 0.00471308,
        0.00451105, 0.00435053, 0.00425725, 0.00416186, 0.00408978]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

### E.coli and P.mirabilis

In [66]:
species_names = ['ecoli', 'kpneumoniae', 'paeruginosa', 'efaecalis', 'abaumanii', 'pmirbalis', 'saureus']
species_titles = ['E.coli', 'K.pneumoniae', 'P.aeruginosa', 'E.faecalis', 'A.baumannii', 'P.mirabilis', 'S.aureus']
No_Ab_Positions = list(range(101, 166))
Ab_Positions = list(range(201, 266))
analysis_dir = root_dir + 'analysis_rev/EXP-22-BZ0313 AST FISH 220629/'
tracking_parameters = {
    'move_iou_thres': 0.15,
    'div_iou_thres' : 0.15,
    'write_dir_names': {
        'channel_stacks': 'blobs',
        'growth_rates': 'growth_rates', # this name will also be used in growth rates
        'fluor_stacks': 'fish_channels',
    },
    'growth_type': 'rolling', # or 'ratio'
    'net_path': '../../saved_models/tracker_model.pth',
    'species_map': {
        'ecoli': ['cy3'],
        'kpneumoniae': ['cy3', 'cy5'],
        'paeruginosa': ['alexa488', 'texasred'],
        'efaecalis': ['alexa488', 'cy5'],
        'abaumanii': ['alexa488', 'cy3'],
        'pmirbalis': ['alexa488'],
        'saureus': ['cy5', 'texasred'],
    },
    'background_channel_no': 20,
    'flip_positions': range(201, 265),
    'num_channels': 42,
    'growth_type': 'ratio'
}
g = growth_rate_pickles(Path(analysis_dir), species_names, species_titles,
                        No_Ab_Positions=No_Ab_Positions, Ab_Positions=Ab_Positions,
                        tracking_parameters=tracking_parameters,
                        antibiotic_concentration=32,
                        antibiotic_name='Nitrofurantoin',
                       n_frames=30)
color_scheme = {
    'ecoli': 'deeppink',
    'pmirbalis': 'green',
}

# This is just fake ignoring 
# You can look at the g variable and see that there are no other species of very little < 30 cells 
g.plot_species_wise_and_pooled(color_scheme=color_scheme,
                               ignore=['paeruginosa','kpneumoniae' , 'efaecalis', 'abaumanii', 'saureus'],
                              species_full_name=species_full_names)

In [67]:
g.species_names

['ecoli',
 'kpneumoniae',
 'paeruginosa',
 'efaecalis',
 'abaumanii',
 'pmirbalis',
 'saureus']

In [68]:
get_raw_data(g, 'ecoli')

ecoli Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.43726617,
        0.37009644, 0.34207545, 0.30581389, 0.28476869, 0.2760315 ,
        0.26798052, 0.2567378 , 0.22626496, 0.21968781, 0.22529807,
        0.20663814, 0.20217349, 0.1852873 , 0.17799283, 0.18421201,
        0.17047568, 0.17492591, 0.16753198, 0.17593993, 0.17302384,
        0.16615176, 0.15469198, 0.15360921, 0.15339442, 0.14979071]),
 array([       nan,        nan,        nan,        nan, 0.01119539,
        0.00845458, 0.00873086, 0.00871125, 0.00833597, 0.00737055,
        0.00638878, 0.00672413, 0.00648884, 0.00632924, 0.00730513,
        0.00659505, 0.00609433, 0.00434385, 0.00403562, 0.00486952,
        0.00427617, 0.00411437, 0.00426943, 0.00429161, 0.00396947,
        0.00446234, 0.00417823, 0.00472202, 0.00438101, 0.00526848]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [69]:
get_raw_data(g, 'pmirbalis')

pmirbalis Normalized growth rates Ab response, sem of growth rate, time


(array([       nan,        nan,        nan,        nan, 0.80625756,
        0.78414257, 0.75194667, 0.71142996, 0.69978646, 0.67624806,
        0.66755213, 0.66456957, 0.64663339, 0.65736506, 0.64330284,
        0.63031931, 0.64343189, 0.61967374, 0.59799307, 0.5885666 ,
        0.57107997, 0.57215816, 0.57212161, 0.56996217, 0.57058765,
        0.5573365 , 0.54035948, 0.52147288, 0.5099002 , 0.50489426]),
 array([       nan,        nan,        nan,        nan, 0.01025219,
        0.01018166, 0.0092702 , 0.01037674, 0.00877992, 0.00817539,
        0.01023393, 0.01029507, 0.00985669, 0.01060993, 0.01037308,
        0.01010773, 0.01061995, 0.0100788 , 0.00741405, 0.00673914,
        0.00587485, 0.00665232, 0.00750024, 0.00669477, 0.00603575,
        0.00567552, 0.00520676, 0.00562769, 0.00596467, 0.00594902]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [70]:
get_raw_data(g, 'pool_Ab')

pool_Ab normalized growth rates, sem of pooled growth rate, time


(array([       nan,        nan,        nan,        nan, 0.62162209,
        0.57672147, 0.54568972, 0.50934209, 0.48361553, 0.46526672,
        0.4558712 , 0.44422716, 0.41773076, 0.41636254, 0.41306541,
        0.39263508, 0.39652339, 0.37701185, 0.3635468 , 0.36337595,
        0.35108955, 0.35680385, 0.35104238, 0.35632946, 0.35547659,
        0.34976478, 0.33807649, 0.3318312 , 0.32772597, 0.32315823]),
 array([       nan,        nan,        nan,        nan, 0.00899231,
        0.00852172, 0.0081723 , 0.00836761, 0.00774041, 0.00711764,
        0.00735155, 0.00733736, 0.0071525 , 0.00729461, 0.00728995,
        0.00689782, 0.0069813 , 0.00634692, 0.00538911, 0.00531131,
        0.00493425, 0.00504766, 0.00530845, 0.00502766, 0.00475005,
        0.00477629, 0.00452907, 0.00466283, 0.00460235, 0.00477607]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))

In [72]:
get_raw_data(g, 'pool_no_Ab')

pool_no_Ab ones, sem of pooled growth rate, time


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]),
 array([       nan,        nan,        nan,        nan, 0.00795531,
        0.00716922, 0.00709855, 0.00635451, 0.00685559, 0.00627076,
        0.00587171, 0.00565116, 0.00522223, 0.00507279, 0.00500058,
        0.00502855, 0.00503974, 0.00500941, 0.00479956, 0.00446411,
        0.00447818, 0.0044691 , 0.0039706 , 0.00382646, 0.00361872,
        0.00360528, 0.00364978, 0.00341434, 0.00345878, 0.00359152]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58]))